## CallbackHandler
- `CallbackHandlers` are objects that implement the `CallbackHandler` interface,
which has a method for each event that can be subscribed to. 
- The `CallbackManager` will call the appropriate method on each handler when the event is triggered.

- The callbacks argument is available on most objects throughout the API (Chains, Models, Tools, Agents, etc.) in two different places:

1. `Constructor callbacks`: defined in the constructor, e.g. LLMChain(callbacks=[handler], tags=['a-tag']), which will be used for all calls made on that object, and will be scoped to that object only, e.g. if you pass a handler to the LLMChain constructor, it will not be used by the Model attached to that chain.
2. `Request callbacks`: defined in the run()/apply() methods used for issuing a request, e.g. chain.run(input, callbacks=[handler]), which will be used for that specific request only, and all sub-requests that it contains (e.g. a call to an LLMChain triggers a call to a Model, which uses the same handler passed in the call() method).

- The `verbose` argument is available on most objects throughout the API (Chains, Models, Tools, Agents, etc.) as a constructor argument, e.g. LLMChain(verbose=True), and it is equivalent to passing a `ConsoleCallbackHandler` to the callbacks argument of that object and all child objects. 
- This is useful for debugging, as it will log all events to the console.

<b>When do you want to use each of these?</b>

- `Constructor callbacks` are most useful for use cases such as logging, monitoring, etc., which are not specific to a single request, but rather to the entire chain. For example, if you want to log all the requests made to an LLMChain, you would pass a handler to the constructor.

- `Request callbacks` are most useful for use cases such as streaming, where you want to stream the output of a single request to a specific websocket connection, or other similar use cases. For example, if you want to stream the output of a single request to a websocket, you would pass a handler to the call() method

In [ ]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage


class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"My custom handler, token: {token}")


# To enable streaming, we pass in `streaming=True` to the ChatModel constructor
# Additionally, we pass in a list with our custom handler
chat = ChatOpenAI(max_tokens=25, streaming=True, callbacks=[MyCustomHandler()])

chat([HumanMessage(content="Tell me a joke")])